In [1]:
from tqdm import tqdm
from utils import set_logger
from dataset import FinedustDataset
from model import FinedustLSTM
from utils import prepare_data
from sklearn.model_selection import train_test_split
import os
import logging
from interpolate import simple_interpolate
import numpy as np
from preprocess import minmax_scaling
from train import train

import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import DataLoader

In [2]:
config = {
    "learning_rate": 1e-3,
    "epochs": 200,
    "batch_size": 32,
    "num_layers": 3,
    "hidden_size": 64,
    "window_size": 56,
    "output_size": 1,
    "dropout": 0.2,
    "patience": 10,
}

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
columns_to_remove = ["CA_TOT", "CA_MID", "STN", "IR", "지점", "위도", "경도"]

region = "Andong"

df = prepare_data(region.lower(), columns_to_remove)
df = simple_interpolate(df, method="linear")
df, scaler = minmax_scaling(df)

dataset = FinedustDataset(df,
                          window_size=config["window_size"],
                          prediction_length=config["output_size"],
                          time_window=1)
train_dataset, val_dataset = train_test_split(dataset, test_size=0.2, random_state=42)
train_loader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=config['batch_size'], shuffle=False)

# LSTM

In [ ]:
total_preds, losses = train(train_dataset,
                            val_dataset,
                            dataset.feature_columns,
                            "Andong", "LSTM", config,
                            device)
# train("Gwangju", "LSTM", config, device, interpolate_fn=simple_interpolate, method="linear")
# train("Seoul", "LSTM", config, device, interpolate_fn=simple_interpolate, method="linear")
# train("Jeonju", "LSTM", config, device, interpolate_fn=simple_interpolate, method="linear")
# train("Daegu", "LSTM", config, device, interpolate_fn=simple_interpolate, method="linear")

# MLP